In [12]:
import numpy as np
import time as tm
import datetime as dt
import tensorflow as tf
from pandas_datareader import data as pdr
import pandas as pd
import snowflake.connector as snow
from snowflake.connector.pandas_tools import write_pandas

# Data preparation
from yahoo_fin import stock_info as yf
import yfinance
from sklearn.preprocessing import MinMaxScaler
from collections import deque

# AI
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [13]:
def get_snowflake_conn():
    conn = snow.connect(
    user="",
    password="",
    account="",
    warehouse="",
    database="",
    schema="")
    return conn

In [14]:
def get_stock_data(stock, date_now, date_3_years_back):
    # LOAD DATA 
    # from yahoo_fin 
    # for 1104 bars with interval = 1d (one day)
    init_df = yf.get_data(
        stock, 
        start_date=date_3_years_back, 
        end_date=date_now, 
        interval='1d')

    return init_df

In [15]:
def PrepareData(init_df, N_STEPS, days):
    df = init_df.copy()
    df['future'] = df['scaled_close'].shift(-days)
    last_sequence = np.array(df[['scaled_close']].tail(days))
    df.dropna(inplace=True)
    sequence_data = []
    sequences = deque(maxlen=N_STEPS)

    for entry, target in zip(df[['scaled_close'] + ['date']].values, df['future'].values):
        sequences.append(entry)
        if len(sequences) == N_STEPS:
            sequence_data.append([np.array(sequences), target])

    last_sequence = list([s[:len(['scaled_close'])] for s in sequences]) + list(last_sequence)
    last_sequence = np.array(last_sequence).astype(np.float32)

    # construct the X's and Y's
    X, Y = [], []
    for seq, target in sequence_data:
        X.append(seq)
        Y.append(target)

    # convert to numpy arrays
    X = np.array(X)
    Y = np.array(Y)

    return df, last_sequence, X, Y

In [16]:
def GetTrainedModel(N_STEPS, x_train, y_train):
    model = Sequential()
    model.add(LSTM(60, return_sequences=True, input_shape=(N_STEPS, len(['scaled_close']))))
    model.add(Dropout(0.3))
    model.add(LSTM(120, return_sequences=False))
    model.add(Dropout(0.3))
    model.add(Dense(20))
    model.add(Dense(1))

    BATCH_SIZE = 8
    EPOCHS = 80

    model.compile(loss='mean_squared_error', optimizer='adam')

    model.fit(x_train, y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            verbose=1)

    model.summary()

    return model

In [17]:
def run_model(stock):

    # Window size or the sequence length, 7 (1 week)
    N_STEPS = 7

    # Lookup steps, 1 is the next day, 3 = after tomorrow
    LOOKUP_STEPS = [1, 2, 3]

    # Current date
    date_now = tm.strftime('%Y-%m-%d')
    date_3_years_back = (dt.date.today() - dt.timedelta(days=1104)).strftime('%Y-%m-%d')

    init_df = get_stock_data(stock, date_now, date_3_years_back)

    # remove columns which our neural network will not use
    init_df = init_df.drop(['open', 'high', 'low', 'adjclose', 'ticker', 'volume'], axis=1)
    # create the column 'date' based on index column
    init_df['date'] = init_df.index 

    # Scale data for ML engine
    scaler = MinMaxScaler()
    init_df['scaled_close'] = scaler.fit_transform(np.expand_dims(init_df['close'].values, axis=1))

#    df, last_sequence, X, Y = PrepareData(init_df, N_STEPS, 3) # 3 days

    # GET PREDICTIONS
    predictions = []

    for step in LOOKUP_STEPS:
        df, last_sequence, x_train, y_train = PrepareData(init_df, N_STEPS, step)
        x_train = x_train[:, :, :len(['close'])].astype(np.float32)

        model = GetTrainedModel(N_STEPS, x_train, y_train)

        last_sequence = last_sequence[-N_STEPS:]
        last_sequence = np.expand_dims(last_sequence, axis=0)
        prediction = model.predict(last_sequence)
        predicted_price = scaler.inverse_transform(prediction)[0][0]

        predictions.append(round(float(predicted_price), 2))

    if bool(predictions) == True and len(predictions) > 0:
        predictions_list = [str(d)+'$' for d in predictions]
        predictions_str = ', '.join(predictions_list)

    # Execute model for the whole history range
    copy_df = init_df.copy()
    y_predicted = model.predict(x_train)
    y_predicted_transformed = np.squeeze(scaler.inverse_transform(y_predicted))
    first_seq = scaler.inverse_transform(np.expand_dims(y_train[:6], axis=1))
    last_seq = scaler.inverse_transform(np.expand_dims(y_train[-3:], axis=1))
    y_predicted_transformed = np.append(first_seq, y_predicted_transformed)
    y_predicted_transformed = np.append(y_predicted_transformed, last_seq)
    copy_df[f'predicted_close'] = y_predicted_transformed

    # Add predicted results to the table
    date_now = dt.date.today()
    date_tomorrow = dt.date.today() + dt.timedelta(days=1)
    date_after_tomorrow = dt.date.today() + dt.timedelta(days=2)

    copy_df.loc[date_now] = [predictions[0], f'{date_now}', 0, 0]
    copy_df.loc[date_tomorrow] = [predictions[1], f'{date_tomorrow}', 0, 0]
    copy_df.loc[date_after_tomorrow] = [predictions[2], f'{date_after_tomorrow}', 0, 0]

    copy_df['date'] = pd.to_datetime(copy_df['date'])
    # Extract date using the 'dt' accessor
    copy_df['date'] = copy_df['date'].dt.date
    copy_df.index = range(0,len(copy_df))
    copy_df['ticker'] = stock

    return copy_df

In [18]:
stocks = ["AAPL", "MSFT", "ADBE", "ABNB", "DASH", "BA", "TSLA", "GOOG", "META", "NVDA"]
conn = get_snowflake_conn()

cur = conn.cursor()

sql = 'drop table BUCKSBUDDY.FINANCE."pred_data"'
cur.execute(sql)
sql = 'drop table BUCKSBUDDY.FINANCE."company_info"'
cur.execute(sql)

info=[]
for i in stocks:
    info.append(yfinance.Ticker(i).info)

company = pd.DataFrame(info)
company.insert(0, "ticker", stocks, True)

write_pandas(conn, company, "company_info", auto_create_table=True)

for i in range(0,len(stocks)):
    result = run_model(stocks[i])
    write_pandas(conn, result, "pred_data", auto_create_table=True)

cur.close()
conn.close()

Epoch 1/80


/Users/anishnavale/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0509
Epoch 2/80
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0075
Epoch 3/80
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0062
Epoch 4/80
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0060
Epoch 5/80
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0058
Epoch 6/80
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0054
Epoch 7/80
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0059
Epoch 8/80
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0044
Epoch 9/80
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0045
Epoch 10/80
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0043
Epoch 11/80
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0035
Epoch 12/80
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0033
Epoch 13/80
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0033
Epoch 14/80
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0034
Epoch 15/80
77/95 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0035

KeyboardInterrupt: 